In [70]:
import networkx as nx
import numpy as np
import pandas as pd 
import operator
import matplotlib.pyplot as plt

In [71]:
nx_graphs_learning = []
nx_graphs_predict = []
agg_graph = nx.MultiGraph()
venue_df_list = []
for i in range(24):
	# with open(f'..\ATX_Networks\sim_day_{i+1}.gexf') as graph_file:
	nx_graphs_learning.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
	venue_dict = nx.get_edge_attributes(nx_graphs_learning[i], "venueid")
	venue_arr = list(venue_dict.items())
	rows = []
	for row in venue_arr:
		rows.append([row[0][0],row[0][1],row[1]])
	venue_df_list.append(pd.DataFrame(rows))


In [72]:
venue_df_list_predict = []
for i in range(24,31):
    nx_graphs_predict.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
    venue_dict = nx.get_edge_attributes(nx_graphs_predict[i-24], "venueid")
    venue_arr = list(venue_dict.items())
    rows = []
    for row in venue_arr:
        rows.append([row[0][0],row[0][1],row[1]])
    venue_df_list_predict.append(pd.DataFrame(rows))

In [73]:
concat_venue_df_learn = pd.concat(venue_df_list)
concat_venue_df_learn.columns = ['source','target','venueid']
learn_edges = concat_venue_df_learn.iloc[:,:2]
concat_venue_df_learn.head()

,source,target,venueid
0,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,440da2cbf964a52091301fe3
1,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,cf4e564f4c8e3aecc9dacafce90cada3eabaff58464e74...,440da2cbf964a52091301fe3
2,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,594a39e074fda9219102eff386f3da1944494aaa18c036...,440da2cbf964a52091301fe3
3,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,a1470e73f1bc4f7c757c49138afd84bd1ad09e9277526a...,440da2cbf964a52091301fe3
4,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,6f8848838f9afe5a7b38c23ad5018179f69e518ee721ba...,440da2cbf964a52091301fe3


In [74]:
concat_venue_df_pred = pd.concat(venue_df_list_predict)
concat_venue_df_pred.columns = ['source','target','venueid']
pred_edges = concat_venue_df_pred.iloc[:,:2]
pred_edges.head()

,source,target
0,5451c4f17a4021464e681b289a42417c2c27cd4c6500b8...,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...
1,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...,e60d9116c902d6fd757c05764783bd7c2578457da97ecb...
2,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...,813a21a92db761f7c5b5752ff4d2ccd675162c8cd0407c...
3,32e829fc1bcd3ab0203e8e12e47a54b8989f84159a0e6a...,8054bde751be29ecde7ade96742833fc45595001a08d1b...
4,32e829fc1bcd3ab0203e8e12e47a54b8989f84159a0e6a...,5e05457c3930c08cde4fe049143384b3e9f3b2cdf2e067...


In [75]:
print(concat_venue_df_learn.shape)
print(concat_venue_df_pred.shape)

(171150, 3)
(59815, 3)


In [76]:
ds_learn = set(map(tuple, learn_edges.values))
ds_pred = set(map(tuple, pred_edges.values))
ds_to_predict = pd.DataFrame(list(ds_pred.difference(ds_learn)))
ds_to_predict.columns=['source', 'target']

In [89]:
ds_outer_join = pd.merge(concat_venue_df_pred,concat_venue_df_learn, on=['source','target'], how='outer', indicator=True).query("_merge == 'left_only'").reset_index(drop=True)

In [90]:
print(concat_venue_df_learn.shape)
print(concat_venue_df_pred.shape)
print(ds_outer_join.shape)
ds_outer_join.head()

(171150, 3)
(59815, 3)
(57032, 5)


,source,target,venueid_x,venueid_y,_merge
0,5451c4f17a4021464e681b289a42417c2c27cd4c6500b8...,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...,4f42f060e4b0a2bdc6d9c3ba,NaN,left_only
1,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...,e60d9116c902d6fd757c05764783bd7c2578457da97ecb...,414f6f00f964a520fc1c1fe3,NaN,left_only
2,9845f55a4a7b78a9c77eb284a4c78a13da532f0b5ee79d...,813a21a92db761f7c5b5752ff4d2ccd675162c8cd0407c...,4cdc3d4122bd721e3916fa47,NaN,left_only
3,32e829fc1bcd3ab0203e8e12e47a54b8989f84159a0e6a...,8054bde751be29ecde7ade96742833fc45595001a08d1b...,54bdad0f498e7bffedf155db,NaN,left_only
4,32e829fc1bcd3ab0203e8e12e47a54b8989f84159a0e6a...,5e05457c3930c08cde4fe049143384b3e9f3b2cdf2e067...,4b5a013cf964a52025a728e3,NaN,left_only


In [91]:
print(ds_to_predict.shape)
print(len(ds_pred))

(56206, 2)
58531


There are 81796 unique colocations in the last 11 days that never happened in the first 20 days. 